<a href="https://colab.research.google.com/github/AzazelSarie/Kaggle/blob/main/SeqNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install category_encoders
!pip install optuna
!pip install tensorflow
!pip install keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 83.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.5/599.5 kB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd
import optuna
import category_encoders as ce
import numpy as np
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.linear_model import Ridge
from sklearn.utils import shuffle

In [ ]:
data = pd.read_csv('/content/train.csv')
data.head()

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


In [ ]:
# Assuming the provided feature engineering steps have already been applied:
data['HP'] = data['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
data['L'] = data['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
data['Other'] = data['engine'].str.replace(r'(\d+\.?\d*)HP', '', regex=True)\
                                        .str.replace(r'(\d+\.?\d*)L', '', regex=True)\
                                        .str.strip()
data['accident_num'] = data['accident'].apply(lambda x: 1 if x == 'At least 1 accident or damage reported' else 0).astype(int)

# Splitting features and target
X = data.drop(columns=['price', 'id', 'accident', 'engine'])  # Dropping unnecessary columns
y = data['price']

X, y = shuffle(X, y)


# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
X.head()

,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,clean_title,HP,L,Other,accident_num
147419,Porsche,911 Carrera S,2023,7751,Gasoline,8-Speed A/T,Green,Black,Yes,443.0,3.0,Flat 6 Cylinder Engine Gasoline Fuel,0
50459,Acura,TLX w/A-Spec Package,2022,21478,Gasoline,2,Lunar Silver Metallic,Ebony,NaN,NaN,NaN,2.0 Liter DOHC Turbo,0
159753,Toyota,Highlander Hybrid Limited Platinum,2019,14700,Hybrid,CVT Transmission,White,Black,Yes,306.0,3.5,V6 Cylinder Engine Gas/Electric Hybrid,0
29062,Chevrolet,Camaro 1SS,2022,17000,Gasoline,6-Speed M/T,Blue,Black,Yes,455.0,6.2,8 Cylinder Engine Gasoline Fuel,0
61457,Honda,Pilot EX-L,2012,194000,Gasoline,A/T,Silver,Gray,Yes,280.0,3.5,V6 Cylinder Engine Gasoline Fuel,1


In [ ]:
# Define numerical and low-cardinality categorical columns
numerical_cols = ['HP', 'L', 'accident_num','model_year']
categorical_low_cardinality_cols = ['fuel_type', 'transmission', 'ext_col', 'int_col', 'Other']

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy = 'mean')),
    ('scaler', MinMaxScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('encoder', ce.LeaveOneOutEncoder())
])

# Define preprocessors for each type of input
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_low_cardinality_cols)
    ])

# Apply preprocessing to the training data
X_train_prepared = preprocessor.fit_transform(X_train, y_train)
X_test_prepared = preprocessor.transform(X_test)

# Convert the sparse matrices to dense NumPy arrays
X_train_prepared_dense = X_train_prepared.toarray() if hasattr(X_train_prepared, 'toarray') else X_train_prepared
X_test_prepared_dense = X_test_prepared.toarray() if hasattr(X_test_prepared, 'toarray') else X_test_prepared


In [ ]:
# Prepare high-cardinality categorical variables for embeddings
X_train_brand = X_train['brand'].astype('category').cat.codes.values.reshape(-1, 1)
X_train_model = X_train['model'].astype('category').cat.codes.values.reshape(-1, 1)
X_test_brand = X_test['brand'].astype('category').cat.codes.values.reshape(-1, 1)
X_test_model = X_test['model'].astype('category').cat.codes.values.reshape(-1, 1)


In [ ]:
print(X_test_prepared_dense.shape)
print(X_test_brand.shape)
print(X_test_model.shape)
print(y_test.shape)

(37707, 9)
(37707, 1)
(37707, 1)
(37707,)


In [ ]:
def rmse(y_true, y_pred):
    # Cast y_true and y_pred to float32 to avoid type mismatch
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # Calculate RMSE
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

# Create the model within the strategy scope
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    # Inputs
    numeric_input = Input(shape=(X_train_prepared.shape[1],), name='input_numeric')  # Preprocessed numerical and one-hot
    brand_input = Input(shape=(1,), name='input_brand')  # Brand input for embedding
    model_input = Input(shape=(1,), name='input_model')  # Model input for embedding

    # Embeddings for high-cardinality categorical features
    brand_embedding = Embedding(input_dim=data['brand'].nunique(), output_dim=8)(brand_input)
    brand_embedding = Flatten()(brand_embedding)
    model_embedding = Embedding(input_dim=data['model'].nunique(), output_dim=16)(model_input)
    model_embedding = Flatten()(model_embedding)

    # Concatenate numerical input, one-hot encoded features, and embeddings
    concatenated = Concatenate()([numeric_input, brand_embedding, model_embedding])

    # Fully connected layers
    # Adjusting the model by increasing neurons, reducing dropout, and reducing L2 regularization
    dense = Dense(256, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.01))(concatenated)
    dense = Dropout(0.4)(dense)
    dense = Dense(128, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.01))(dense)
    dense = Dropout(0.4)(dense)
    dense = Dense(64, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=l2(0.01))(dense)
    dense = Dropout(0.4)(dense)
    output = Dense(1)(dense)  # Output layer for regression


    # Build and compile the model using RMSE as the loss function
    model = Model(inputs=[numeric_input, brand_input, model_input], outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss=rmse, metrics=[rmse])

# Create a callback to reduce learning rate when validation loss plateaus
lr_scheduler = ReduceLROnPlateau(monitor='val_rmse', factor=0.5, patience=5, min_lr=1e-6)

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_rmse', patience=10, restore_best_weights=True, verbose=1)

# Train the model using the dense input data
history = model.fit([X_train_prepared_dense, X_train_brand, X_train_model], y_train,
                    epochs=400, batch_size=128, validation_split=0.2, callbacks=[lr_scheduler, early_stopping], verbose=2)

# Evaluate the model on the test set
test_loss, test_rmse = model.evaluate([X_test_prepared_dense, X_test_brand, X_test_model], y_test, verbose=2)

# Print test loss and RMSE
print(f'Test Loss: {test_loss}, Test RMSE: {test_rmse}')


Epoch 1/400
943/943 - 6s - loss: 58997.4258 - rmse: 58995.8125 - val_loss: 60929.8789 - val_rmse: 60875.7773 - lr: 0.0010 - 6s/epoch - 6ms/step
Epoch 2/400
943/943 - 4s - loss: 57640.3008 - rmse: 57623.5781 - val_loss: 60748.4414 - val_rmse: 60692.7461 - lr: 0.0010 - 4s/epoch - 5ms/step
Epoch 3/400
943/943 - 4s - loss: 57506.0312 - rmse: 57493.4805 - val_loss: 58927.7109 - val_rmse: 58873.2383 - lr: 0.0010 - 4s/epoch - 5ms/step
Epoch 4/400
943/943 - 4s - loss: 57211.5273 - rmse: 57203.1133 - val_loss: 58654.3203 - val_rmse: 58600.5586 - lr: 0.0010 - 4s/epoch - 5ms/step
Epoch 5/400
943/943 - 4s - loss: 57007.8828 - rmse: 56994.1602 - val_loss: 59021.3320 - val_rmse: 58963.7695 - lr: 0.0010 - 4s/epoch - 5ms/step
Epoch 6/400
943/943 - 4s - loss: 57297.1328 - rmse: 57284.1523 - val_loss: 58611.9023 - val_rmse: 58554.3320 - lr: 0.0010 - 4s/epoch - 5ms/step
Epoch 7/400
943/943 - 4s - loss: 57400.3242 - rmse: 57383.8047 - val_loss: 58020.2734 - val_rmse: 57962.0703 - lr: 0.0010 - 4s/epoch - 5

In [ ]:
test_data = pd.read_csv('/content/test.csv')

# Assuming the provided feature engineering steps have already been applied:
test_data['HP'] = test_data['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float)
test_data['L'] = test_data['engine'].str.extract(r'(\d+\.?\d*)L').astype(float)
test_data['Other'] = test_data['engine'].str.replace(r'(\d+\.?\d*)HP', '', regex=True)\
                                        .str.replace(r'(\d+\.?\d*)L', '', regex=True)\
                                        .str.strip()
test_data['accident_num'] = test_data['accident'].apply(lambda x: 1 if x == 'At least 1 accident or damage reported' else 0).astype(int)
test_data = test_data.drop(columns=['accident', 'engine'])

# Apply the preprocessing without converting to dense to avoid memory issues
X_test_prepared_sparse = preprocessor.transform(test_data)

# Generate predictions using the trained model (handle sparse data without converting to dense)
X_test_brand = test_data['brand'].astype('category').cat.codes.values.reshape(-1, 1)
X_test_model = test_data['model'].astype('category').cat.codes.values.reshape(-1, 1)

# Generate predictions using the trained model
predictions = model.predict([X_test_prepared_sparse, X_test_brand, X_test_model])

3928/3928 [==============================] - 8s 2ms/step


In [ ]:
# Attach the predictions to the original test data
test_data['price'] = predictions.astype(int)

# Display the test data with the predicted prices
test_data.head()

,id,brand,model,model_year,milage,fuel_type,transmission,ext_col,int_col,clean_title,HP,L,Other,accident_num,price
0,188533,Land,Rover LR2 Base,2015,98000,Gasoline,6-Speed A/T,White,Beige,Yes,240.0,2.0,4 Cylinder Engine Gasoline Fuel,0,21193
1,188534,Land,Rover Defender SE,2020,9142,Hybrid,8-Speed A/T,Silver,Black,Yes,395.0,3.0,Straight 6 Cylinder Engine Gasoline/Mild Elect...,0,61276
2,188535,Ford,Expedition Limited,2022,28121,Gasoline,10-Speed Automatic,White,Ebony,NaN,NaN,3.5,V6 24V PDI DOHC Twin Turbo,0,55686
3,188536,Audi,A6 2.0T Sport,2016,61258,Gasoline,Automatic,Silician Yellow,Black,NaN,NaN,NaN,2.0 Liter TFSI,0,44792
4,188537,Audi,A6 2.0T Premium Plus,2018,59000,Gasoline,A/T,Gray,Black,Yes,252.0,2.0,4 Cylinder Engine Gasoline Fuel,0,18778


In [ ]:
test_data[['id', 'price']].to_csv('predictions.csv', index=False)